In [0]:
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import SimpleRNN
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
import pandas as pd
from string import punctuation

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# keras로 Neural Network 모델링 (간략 복습)

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [0]:
# model.add(SimpleRNN(hidden_size))
model = Sequential()
model.add(Dense(3, input_dim=4, activation="softmax"))
# Sequential을 사용하는 방법  --  Sequential API

In [0]:
# FC FFNN (Fully Connected Feed Forward Neural Network)

from keras.layers import Input, Dense
from keras.models import Model

inputs = Input(shape=(10,))    # 입력이 10개인 층 구성
# input으로도 layer를 구성할 수 있다  --  functional API
hid1 = Dense(32, activation="relu")(inputs)
hid2 = Dense(32, activation="relu")(hid1)
output = Dense(1, activation="sigmoid")(hid2)
model = Model(inputs=inputs, outputs=output)

model.compile((optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]))
# model.fit(입력데이터, y레이블)

# keras를 이용해서 RNN 계층 구성

In [0]:
inputs = Input(shape=(30, 1))    # 30: time-step
lstm_layer = LSTM(10)(inputs)
x = Dense(10, activation="relu")(lstm_layer)
output = Dense(1, activation="sigmoid")(x)
Model(inputs=inputs, outputs=output)

# keras에서 RNN 구현

## Size 알아보기

In [0]:
from keras.layers import SimpleRNN

model = Sequential()
# model.add(SimpleRNN(3, input_shape=(2, 10)))
# input shape에서 2는 time-step, 10은 input_dim
model.add(SimpleRNN(3, input_length=2, input_dim=10))    # 주석처리된 위 코드와 결과 동일
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(3, input_shape=(2, 10))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [0]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
# 8: batch size, 2: time step, 10: input dim
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [0]:
import numpy as np

timesteps = 10    # 시점    # 예: 문장을 구성하는 단어의 길이 or 단어를 구성하는 문자의 갯수
# 예: 주식예측모델에서 timestep을 7로 줬다면, 7일 동안의 주가 흐름을 기초로 오늘 주가를 예측하겠다는 의미
input_dim = 4    # ㅇ    # 코퍼스 내에 저장된 단어의 개수
hidden_size = 8    # 은닉 상태의 크기 (셀)

In [0]:
inputs = np.random.random((timesteps, input_dim))
print(inputs)
print(inputs.shape)

[[0.50361742 0.6854732  0.75433197 0.97514154]
 [0.75948393 0.12375935 0.44369472 0.05920386]
 [0.13232218 0.43068148 0.05205566 0.45104022]
 [0.48971125 0.19903326 0.54107725 0.91523035]
 [0.92045716 0.39150321 0.60234521 0.97463573]
 [0.2885812  0.77441747 0.88442858 0.73750652]
 [0.85684984 0.44023406 0.93062836 0.6236884 ]
 [0.40664044 0.25576602 0.56721063 0.57179129]
 [0.05414876 0.96244545 0.27716503 0.5195851 ]
 [0.14523624 0.14349408 0.65498258 0.05214904]]
(10, 4)


In [0]:
# 초기 은닉 상태를 0으로 초기화

hidden_state_t = np.zeros((hidden_size,))
hidden_state_t

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
# 출력을 고려해라
# ht = tanh(wx*xt + wh*ht-1 + b)
# (8,1) = tanh((8,4)*(4,1) + (8,8)*(8,1) + 8)

# 가중치

# wx:입력에 대한 가중치
# wx: (은닉 상태의 크기 * 입력의 차원)
wx = np.random.random((hidden_size, input_dim))    # (8,4)

# wh: 은닉 상태 가중치
# wh: (은닉 상태의 크기 * 은닉 상태의 크기)
wh = np.random.random((hidden_size, hidden_size))    # (8,8)

# b: bias
# b: 은닉 상태의 크기
b = np.random.random((hidden_size))    # 8

print(np.shape(wx))

(8, 4)


In [0]:
# 메모리 셀 동작
for input_t in inputs:
  print(input_t)

[0.50361742 0.6854732  0.75433197 0.97514154]
[0.75948393 0.12375935 0.44369472 0.05920386]
[0.13232218 0.43068148 0.05205566 0.45104022]
[0.48971125 0.19903326 0.54107725 0.91523035]
[0.92045716 0.39150321 0.60234521 0.97463573]
[0.2885812  0.77441747 0.88442858 0.73750652]
[0.85684984 0.44023406 0.93062836 0.6236884 ]
[0.40664044 0.25576602 0.56721063 0.57179129]
[0.05414876 0.96244545 0.27716503 0.5195851 ]
[0.14523624 0.14349408 0.65498258 0.05214904]


In [0]:
total_hidden_states = []

# 메모리 셀 동작

for input_t in inputs:
  output_t = np.tanh(np.dot(wx, input_t) + np.dot(wh, hidden_state_t) + b)
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t
  print(output_t)
print ("=" * 10)
print(np.shape(total_hidden_states))
print(total_hidden_states)

[0.9817166  0.979308   0.64551417 0.9819094  0.88037155 0.98816619
 0.9659939  0.94541546]
[0.997236   0.99995553 0.99849574 0.99998902 0.99838873 0.99966428
 0.99986926 0.999791  ]
[0.99854033 0.99995045 0.99930432 0.99999069 0.99945406 0.99961202
 0.99964088 0.99993114]
[0.99961083 0.99998232 0.99953196 0.99999789 0.99974545 0.9998085
 0.9999243  0.99994051]
[0.99984372 0.99999351 0.99963527 0.99999945 0.99981921 0.99993438
 0.99997743 0.99996363]
[0.99975296 0.99998936 0.99977065 0.99999901 0.99978437 0.99992765
 0.9999682  0.9999792 ]
[0.99976036 0.99999395 0.99971805 0.9999993  0.99974299 0.99993762
 0.99998615 0.99995971]
[0.99926212 0.99997785 0.99952673 0.9999963  0.99960324 0.99976957
 0.99991312 0.99992888]
[0.9994151  0.99997497 0.99963161 0.99999714 0.999659   0.99987317
 0.99985339 0.99997823]
[0.99712773 0.99995381 0.99945558 0.99998445 0.99910207 0.99947602
 0.99984072 0.99986285]
(10, 8)
[[0.9817165987938186, 0.9793080020082924, 0.6455141662780628, 0.9819093973898296, 0

# keras로 구현하는 RNN을 이용한 텍스트 생성

## 데이터 전처리

In [0]:
text = """
  멀티캠퍼스에 있는 교육생들이 공부하고 있다\n
  교육생들은 딥러닝을 하고 있다\n
  결석한 교육생도 있고 지각하는 교육생도 있다\n
"""
print(text)


  멀티캠퍼스에 있는 교육생들이 공부하고 있다

  교육생들은 딥러닝을 하고 있다

  결석한 교육생도 있고 지각하는 교육생도 있다




In [0]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])
print(t.texts_to_sequences([text]))    # 인덱스 출력
print(t.word_index)

[[3, 4, 5, 6, 1, 7, 8, 9, 1, 10, 2, 11, 12, 2, 1]]
{'있다': 1, '교육생도': 2, '멀티캠퍼스에': 3, '있는': 4, '교육생들이': 5, '공부하고': 6, '교육생들은': 7, '딥러닝을': 8, '하고': 9, '결석한': 10, '있고': 11, '지각하는': 12}


In [0]:
vocab_size = len(t.word_index)+1    # 단어 집합 크기: 13
# vocab_size에 1을 더하는 이유는 나중에 이 vocab_size를 가지고 원핫인코딩을 할 것이기 때문이다.
# word index는 1부터 시작하는데, 원핫인코딩을 하면 0부터 시작한다.
# 그러므로 원핫인코딩을 할 때 num_classes를 word_index의 length에 맞추면 맨 마지막 인덱스는 잘리게 된다
# 모든 인덱스를 다 나오게 하려면 0의 자리를 포함하기 위해 length에 1을 더해줘야 한다.

In [0]:
# 예측 1: 다음에 올 단어 예측
# 예측 2: 다음에 올 단어 n개 예측 (입력: "멀티캠퍼스에", 단어수:2, 출력결과: "있는 교육생들이")

In [0]:
for line in text.split("\n"):
  print(line)

print ("=" * 10)

for line in text.split("\n"):
  print(t.texts_to_sequences([line])[0])

print ("=" * 10)

for line in text.split("\n"):
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    print(sequence)


  멀티캠퍼스에 있는 교육생들이 공부하고 있다

  교육생들은 딥러닝을 하고 있다

  결석한 교육생도 있고 지각하는 교육생도 있다


[]
[3, 4, 5, 6, 1]
[]
[7, 8, 9, 1]
[]
[10, 2, 11, 12, 2, 1]
[]
[]
[3, 4]
[3, 4, 5]
[3, 4, 5, 6]
[3, 4, 5, 6, 1]
[7, 8]
[7, 8, 9]
[7, 8, 9, 1]
[10, 2]
[10, 2, 11]
[10, 2, 11, 12]
[10, 2, 11, 12, 2]
[10, 2, 11, 12, 2, 1]


In [0]:
sequences = []

for line in text.split("\n"):
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print(len(sequences))
print(sequences)

# 학습을 시키려면 동일한 길이로 통일시켜야 한다
print(max(len(s) for s in sequences))    # 최대 길이 = 6
max_len = max(len(s) for s in sequences)

12
[[3, 4], [3, 4, 5], [3, 4, 5, 6], [3, 4, 5, 6, 1], [7, 8], [7, 8, 9], [7, 8, 9, 1], [10, 2], [10, 2, 11], [10, 2, 11, 12], [10, 2, 11, 12, 2], [10, 2, 11, 12, 2, 1]]
6


In [0]:
from keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
sequences

array([[ 0,  0,  0,  0,  3,  4],
       [ 0,  0,  0,  3,  4,  5],
       [ 0,  0,  3,  4,  5,  6],
       [ 0,  3,  4,  5,  6,  1],
       [ 0,  0,  0,  0,  7,  8],
       [ 0,  0,  0,  7,  8,  9],
       [ 0,  0,  7,  8,  9,  1],
       [ 0,  0,  0,  0, 10,  2],
       [ 0,  0,  0, 10,  2, 11],
       [ 0,  0, 10,  2, 11, 12],
       [ 0, 10,  2, 11, 12,  2],
       [10,  2, 11, 12,  2,  1]], dtype=int32)

In [0]:
x = sequences[:, :-1]
y = sequences[:, -1]
x

array([[ 0,  0,  0,  0,  3],
       [ 0,  0,  0,  3,  4],
       [ 0,  0,  3,  4,  5],
       [ 0,  3,  4,  5,  6],
       [ 0,  0,  0,  0,  7],
       [ 0,  0,  0,  7,  8],
       [ 0,  0,  7,  8,  9],
       [ 0,  0,  0,  0, 10],
       [ 0,  0,  0, 10,  2],
       [ 0,  0, 10,  2, 11],
       [ 0, 10,  2, 11, 12],
       [10,  2, 11, 12,  2]], dtype=int32)

In [0]:
from keras.utils import to_categorical

In [0]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

## 모델 생성 및 학습

In [0]:
from keras.layers import *

In [0]:
model = Sequential()

# Embedding: sparse vector를 dense vector로
# keras.layers.Embedding(input_dim, output_dim, input_length=None)
model.add(Embedding(vocab_size, 10, input_length=max_len-1))    # max_len-1인 이유는 마지막 1열을 잘라서 x를 만들었으니까
# model.add(SimpleRNN(출력))
model.add(SimpleRNN(32))

# FC layer -> softmax
model.add(Dense(vocab_size, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x, y, epochs=300, verbose=2)

W0823 04:43:54.888712 140590976526208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 04:43:54.917365 140590976526208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0823 04:43:55.072325 140590976526208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0823 04:43:55.301517 140590976526208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Epoch 1/300
 - 1s - loss: 2.5909 - acc: 0.1667
Epoch 2/300
 - 0s - loss: 2.5818 - acc: 0.1667
Epoch 3/300
 - 0s - loss: 2.5716 - acc: 0.1667
Epoch 4/300
 - 0s - loss: 2.5609 - acc: 0.0833
Epoch 5/300
 - 0s - loss: 2.5501 - acc: 0.0833
Epoch 6/300
 - 0s - loss: 2.5392 - acc: 0.1667
Epoch 7/300
 - 0s - loss: 2.5277 - acc: 0.0833
Epoch 8/300
 - 0s - loss: 2.5164 - acc: 0.1667
Epoch 9/300
 - 0s - loss: 2.5049 - acc: 0.1667
Epoch 10/300
 - 0s - loss: 2.4930 - acc: 0.1667
Epoch 11/300
 - 0s - loss: 2.4808 - acc: 0.1667
Epoch 12/300
 - 0s - loss: 2.4682 - acc: 0.2500
Epoch 13/300
 - 0s - loss: 2.4550 - acc: 0.2500
Epoch 14/300
 - 0s - loss: 2.4413 - acc: 0.2500
Epoch 15/300
 - 0s - loss: 2.4271 - acc: 0.2500
Epoch 16/300
 - 0s - loss: 2.4122 - acc: 0.2500
Epoch 17/300
 - 0s - loss: 2.3966 - acc: 0.2500
Epoch 18/300
 - 0s - loss: 2.3804 - acc: 0.2500
Epoch 19/300
 - 0s - loss: 2.3636 - acc: 0.2500
Epoch 20/300
 - 0s - loss: 2.3461 - acc: 0.2500
Epoch 21/300
 - 0s - loss: 2.3280 - acc: 0.2500
E

## 예측

In [0]:
def word_gen(model, t, curr_word, num):    # 모델, 토크나이저, 입력단어, 갯수
  ini_word = curr_word
  sent = ""
  for _ in range(num):
    # print(t.word_index)
    # print(t.texts_to_sequences([curr_word]))
    encoded = t.texts_to_sequences([curr_word])[0]    # [3]  --  "멀티캠퍼스에"의 인덱스인 3을 출력
    # print(encoded)
    # shape 맞추기 위한 padding
    # print(pad_sequences([encoded], maxlen=5, padding="pre"))    # 5는 입력 데이터(x) 길이
    encoded = pad_sequences([encoded], maxlen=5, padding="pre")
    # 예측
    res = model.predict_classes(encoded)
    # print(res)    # 4
    for word, index in t.word_index.items():
      if res==index:    # 동일한 단어를 찾았다면
        break
    curr_word = curr_word + " " + word
    sent = sent + " " + word
  sent = ini_word + sent
  return sent

print(word_gen(model, t, "멀캠", 10))    # "멀티캠퍼스에" 라는 단어 뒤 3개의 단어 예측

멀캠 교육생도 교육생도 있고 지각하는 교육생도 있다 있고 지각하는 있다 있다


# NYT 기사 관련 자료로 복습

## 데이터 전처리

In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/190000 멀티캠퍼스/Data/ArticlesApril2018.csv")
df

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
5,5adf341f068401528a2aa5e8,1233,By NICHOLAS FANDOS and MICHAEL D. SHEAR,article,Unknown,"['United States Politics and Government', 'Vet...",68,Washington,0,2018-04-24 13:41:49,Politics,The Senate Veterans Affairs Committee is exami...,The New York Times,News,https://www.nytimes.com/2018/04/24/us/politics...
6,5adf2a0f068401528a2aa5c8,871,By ADAM NAGOURNEY and CHARLES McDERMID,article,Unknown,[],0,NYTNow,0,2018-04-24 12:58:53,Unknown,Tuesday: Rent control appears headed to the ba...,The New York Times,briefing,https://www.nytimes.com/2018/04/24/us/californ...
7,5adf26cf068401528a2aa5c1,938,By NIRAJ CHOKSHI,article,Unknown,"['Presidential Election of 2016', 'Proceedings...",68,Express,0,2018-04-24 12:45:00,Politics,"A new study suggests that the white, Christian...",The New York Times,News,https://www.nytimes.com/2018/04/24/us/politics...
8,5adf2108068401528a2aa5b3,730,By JOHN SCHWARTZ,article,How a Bag of Texas Dirt Became a Times Tradition,"['Texas', 'Soil', 'Pregnancy and Childbirth', ...",68,Insider,2,2018-04-24 12:20:21,Unknown,What can you do when you want your child to be...,The New York Times,News,https://www.nytimes.com/2018/04/24/insider/how...
9,5adedaa8068401528a2aa4e6,482,By NATALIE PROULX,article,Is School a Place for Self-Expression?,[],66,Learning,0,2018-04-24 11:21:04,Unknown,Are you encouraged to question and articulate ...,The New York Times,News,https://www.nytimes.com/2018/04/24/learning/is...


In [0]:
print(df.columns)
print(len(df.columns))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
15


In [0]:
print(len(df))    # 1324
df["headline"].isnull().values.any()

1324


False

In [0]:
headline = []
headline.extend(list(df.headline.values))
headline

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Unknown',
 'Unknown',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'Unknown',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafa

### 노이즈(unknown) 제거

In [0]:
headline = [n for n in headline if n != "Unknown"]
print(len(headline))
print(headline)

1214
['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell', 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.', 'The New Noma, Explained', 'How a Bag of Texas Dirt  Became a Times Tradition', 'Is School a Place for Self-Expression?', 'Commuter Reprogramming', 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.', 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed', 'Chain Reaction', 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.', 'In Berlin, artists find a home', 'The Right Stuff', 'Jimmy Carter Knows What North Korea Wants', 'The Truth Is Out There', 'New Jersey Ruling Could Reignite Battle Over Church-State Separation', 'Procrastinating', 'Word + Quiz: dilatory', 'My Life-Threatening Bout With E. Coli Food Poisoning', 'Choosing Brexit, a Town Yearned for Its Seafaring Past, and Muddied Its Future', 'A Quote Disproved', 'Hot Stuff Turns Cold', 'At the Top, The Pay Gap May

### 점 제거, 소문자 변환 등

In [0]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [0]:
def mypreprocessing(s):    # 전처리 된 문장을 리턴
  return "".join(ch for ch in s if ch not in punctuation).lower()    # for ch in s: 주어진 문장에서 글자를 한 글자씩 뽑아서 처리

text = [mypreprocessing(x) for x in headline]    # 변환되기 전 원문장을 입력 -> 전처리된 문장 출력
print(text)
print(len(text))

['former nfl cheerleaders’ settlement offer 1 and a meeting with goodell', 'epa to unveil a new rule its effect less science in policymaking', 'the new noma explained', 'how a bag of texas dirt  became a times tradition', 'is school a place for selfexpression', 'commuter reprogramming', 'ford changed leaders looking for a lift it’s still looking', 'romney failed to win at utah convention but few believe he’s doomed', 'chain reaction', 'he forced the vatican to investigate sex abuse now he’s meeting with pope francis', 'in berlin artists find a home', 'the right stuff', 'jimmy carter knows what north korea wants', 'the truth is out there', 'new jersey ruling could reignite battle over churchstate separation', 'procrastinating', 'word  quiz dilatory', 'my lifethreatening bout with e coli food poisoning', 'choosing brexit a town yearned for its seafaring past and muddied its future', 'a quote disproved', 'hot stuff turns cold', 'at the top the pay gap may be gone', 'years of bizarre behav

### vocabulary 작성 (단어 집합)

In [0]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index)
len(t.word_index)

{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'on': 8, 'is': 9, 'with': 10, 'trump': 11, 'as': 12, 'at': 13, 'new': 14, 'from': 15, 'how': 16, 'it': 17, 'an': 18, 'be': 19, 'that': 20, 'season': 21, 'us': 22, 'you': 23, 'what': 24, 'episode': 25, 'can': 26, 'your': 27, 'he': 28, 'not': 29, 'his': 30, 'are': 31, 'teaching': 32, 'out': 33, 'was': 34, 'no': 35, 'by': 36, 'trump’s': 37, 'has': 38, 'war': 39, 'over': 40, 'may': 41, 'into': 42, 'more': 43, 'now': 44, 'why': 45, 'who': 46, 'about': 47, 'recap': 48, 'activities': 49, '1': 50, 'we': 51, 'do': 52, 'women': 53, 'when': 54, 'syria': 55, 'trade': 56, 'it’s': 57, '2': 58, 'or': 59, 'will': 60, 'i': 61, 'have': 62, 'just': 63, 'but': 64, 'home': 65, 'up': 66, 'this': 67, 'one': 68, 'off': 69, 'house': 70, 'president': 71, 'gop': 72, 'our': 73, 'long': 74, 'facebook': 75, 'case': 76, 'life': 77, 'end': 78, 'some': 79, 'power': 80, 'say': 81, 'white': 82, '‘the': 83, 'after': 84, 'its': 85, 'still': 86, 'right': 87,

3619

In [0]:
vocab_size = len(t.word_index)+1    # 단어 집합의 크기: 3620

In [0]:
sequences = []
for line in text:    # 1214개의 line
  encoded = t.texts_to_sequences([line])[0]    # 주어진 문장의 단어들을 tokenize된 인덱스로 치환    # 2차원이므로 1차원으로 내려줌
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

[[95, 263], [95, 263, 1100], [95, 263, 1100, 1101], [95, 263, 1100, 1101, 572], [95, 263, 1100, 1101, 572, 50], [95, 263, 1100, 1101, 572, 50, 7], [95, 263, 1100, 1101, 572, 50, 7, 2], [95, 263, 1100, 1101, 572, 50, 7, 2, 365], [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10], [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102], [96, 3], [96, 3, 1103], [96, 3, 1103, 2], [96, 3, 1103, 2, 14], [96, 3, 1103, 2, 14, 573], [96, 3, 1103, 2, 14, 573, 85], [96, 3, 1103, 2, 14, 573, 85, 1104], [96, 3, 1103, 2, 14, 573, 85, 1104, 366], [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367], [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5]]


In [0]:
max_len = max(len(s) for s in sequences)
max_len

24

In [0]:
sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
sequences

array([[   0,    0,    0, ...,    0,   95,  263],
       [   0,    0,    0, ...,   95,  263, 1100],
       [   0,    0,    0, ...,  263, 1100, 1101],
       ...,
       [   0,    0,    0, ...,    9, 3619,  110],
       [   0,    0,    0, ..., 3619,  110,    2],
       [   0,    0,    0, ...,  110,    2, 1014]], dtype=int32)

In [0]:
x = sequences[:, :-1]
y = sequences[:, -1]
print(x)
print(y)

[[   0    0    0 ...    0    0   95]
 [   0    0    0 ...    0   95  263]
 [   0    0    0 ...   95  263 1100]
 ...
 [   0    0    0 ...  216    9 3619]
 [   0    0    0 ...    9 3619  110]
 [   0    0    0 ... 3619  110    2]]
[ 263 1100 1101 ...  110    2 1014]


In [0]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## 모델 생성 및 학습

In [120]:
model = Sequential()

model.add(Embedding(vocab_size, 128, input_length=max_len-1))
model.add(SimpleRNN(64))

model.add(Dense(vocab_size, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x, y, epochs=100, verbose=2)    # verbose는 training progress를 '보여주는' 방식 (https://stackoverflow.com/questions/47902295)

Epoch 1/100
 - 6s - loss: 7.7011 - acc: 0.0266
Epoch 2/100
 - 5s - loss: 7.0724 - acc: 0.0380
Epoch 3/100
 - 5s - loss: 6.7867 - acc: 0.0497
Epoch 4/100
 - 5s - loss: 6.4881 - acc: 0.0579
Epoch 5/100
 - 5s - loss: 6.1721 - acc: 0.0700
Epoch 6/100
 - 5s - loss: 5.8317 - acc: 0.0868
Epoch 7/100
 - 5s - loss: 5.4892 - acc: 0.1136
Epoch 8/100
 - 5s - loss: 5.1485 - acc: 0.1404
Epoch 9/100
 - 5s - loss: 4.8119 - acc: 0.1763
Epoch 10/100
 - 5s - loss: 4.4721 - acc: 0.2256
Epoch 11/100
 - 5s - loss: 4.1396 - acc: 0.2920
Epoch 12/100
 - 5s - loss: 3.8138 - acc: 0.3473
Epoch 13/100
 - 5s - loss: 3.4975 - acc: 0.4171
Epoch 14/100
 - 5s - loss: 3.1948 - acc: 0.4855
Epoch 15/100
 - 5s - loss: 2.9010 - acc: 0.5546
Epoch 16/100
 - 5s - loss: 2.6527 - acc: 0.6008
Epoch 17/100
 - 5s - loss: 2.3746 - acc: 0.6605
Epoch 18/100
 - 5s - loss: 2.1312 - acc: 0.7046
Epoch 19/100
 - 5s - loss: 1.9108 - acc: 0.7443
Epoch 20/100
 - 5s - loss: 1.7125 - acc: 0.7753
Epoch 21/100
 - 5s - loss: 1.5347 - acc: 0.8039
E

## 예측

In [121]:
def article_gen(model, t, curr_word, num):
  init_word = curr_word
  sent = ""
  for _ in range(num):
    encoded = t.texts_to_sequences([curr_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1, padding="pre")
    # predict 는 probability를 predict_class 는 label 을 제공한다 (https://mmresult.tistory.com/entry/predict-%EC%99%80-predictclass-%EC%99%80%EC%9D%98-%EC%B0%A8%EC%9D%B4%EC%A0%90)
    res = model.predict_classes(encoded)    # to_categorical로 원핫인코딩한 결과가 아닌, 디코딩된 결과 (단어의 인덱스 넘버)가 res로 출력된다
    for word, index in t.word_index.items():
      if index == res:
        break
    curr_word = curr_word + " " + word    # for문에서 다음 턴에 들어갈 input
    sent = sent + " " + word    # for문을 통해 누적되는 outputs
  sent = init_word + sent    # 최종 output
  return sent

print(article_gen(model, t, "how", 10))

how to make a crossword puzzle sam trabucco try merging a
